<a href="https://colab.research.google.com/github/mangalm96/explainablenlp-info259/blob/main/Captum_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### installations

In [2]:
!pip install transformers
!pip install captum

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 39.3 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 895 kB 60.6 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.4 MB 5.3 MB/s 


In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from transformers import BertTokenizer, BertForQuestionAnswering, BertConfig
from transformers import AutoTokenizer, AdamW, AutoModel


from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
device

device(type='cpu')

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### start

In [11]:
class BERTStance(nn.Module):
    def __init__(self, num_stances=4):
        super(BERTStance, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/bertweet-base")
        self.drop = nn.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_stances)
    def forward(self, text, att_mask, token_type):
        _, pooled = self.bert(text, attention_mask=att_mask, token_type_ids=token_type)
        return self.classifier(self.drop(pooled))
        


In [12]:
model_path = "/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/saves/wtwt_CI_ESRX_target/model.pt"
modelb = BERTStance(4)
embedding_size = modelb.bert.embeddings.word_embeddings.weight.size(1)
new_embeddings = torch.FloatTensor(3, embedding_size).uniform_(-0.1, 0.1)
new_embedding_weight = torch.cat((modelb.bert.embeddings.word_embeddings.weight.data,new_embeddings), 0)
modelb.bert.embeddings.word_embeddings.weight.data = new_embedding_weight
print("Embedding Shape:", modelb.bert.embeddings.word_embeddings.weight.data.size())

modelb.load_state_dict(torch.load(model_path))
modelb = modelb.to(device)

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding Shape: torch.Size([64004, 768])


RuntimeError: ignored

In [ ]:
model_path = '<PATH-TO-SAVED-MODEL>'

# load model
model = BertForQuestionAnswering.from_pretrained(model_path)